# AGENT RNN PAR OLIVIER

In [1]:
!pip install tqdm


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Préparons l'environnement

In [1]:
from importlib.metadata import version
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import numpy as np
import math
from dataclasses import dataclass
from matplotlib import pyplot as plt
import time
import os
from ipywidgets import Output
from tqdm.notebook import trange, tqdm
# Import necessary libraries
from matplotlib.colors import ListedColormap, BoundaryNorm
from IPython.display import display

# Pour torch si vous avez un GPU
# device = "cpu" if not torch.cuda.is_available() else "cuda"
device = "cpu" # Pour forcer l'utilisation du CPU

Les actions

In [2]:
FORWARD = 2
TURN_LEFT = 3
TURN_RIGHT = 4
FEEL_FRONT = 5
FEEL_LEFT = 6  # Non utilisé
FEEL_RIGHT = 7  # Non utilisé

Le Small Loop

In [3]:
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3
FEELING = 2
BUMPING = 3

class SmallLoop():
    def __init__(self, poX, poY, direction):
        self.grid = np.array([
                [1, 1, 1, 1, 1],
                [1, 0, 0, 0, 1],
                [1, 0, 1, 0, 1],
                [1, 0, 0, 0, 1],
                [1, 1, 1, 1, 1]
        ])
        self.maze = self.grid.copy()
        self.poX = poX
        self.poY = poY
        self.direction = direction
        self.cmap = ListedColormap(['white', 'green', 'yellow', 'red'])
        self.norm = BoundaryNorm([-0.5, 0.5, 1.5, 2.5, 3.5], self.cmap.N)

    def outcome(self, action):
        # print('before:', self.agent_position.strPosition(), action_dcit[action])
        self.maze[:,:] = self.grid
        result = 0
        
        if action == FORWARD:  # move forward
            # print('the action is move forward')
            # print(str(self.position.pointX)+': '+ str(self.position.pointY)+ ' ' +self.direction, action)
        
            if self.direction == LEFT:
                if self.maze[self.poX][self.poY - 1] == 0:
                    self.poY -= 1
                else:
                    self.maze[self.poX][self.poY - 1] = BUMPING
                    result = 1
            elif self.direction == DOWN:
                if self.maze[self.poX + 1][self.poY] == 0:
                    self.poX += 1
                else:
                    self.maze[self.poX + 1][self.poY] = BUMPING
                    result = 1
            elif self.direction == RIGHT:
                if self.maze[self.poX][self.poY + 1] == 0:
                    self.poY += 1
                else:
                    self.maze[self.poX][self.poY + 1] = BUMPING
                    result = 1
            elif self.direction == UP:
                if self.maze[self.poX - 1][self.poY] == 0:
                    self.poX -= 1
                else:
                    self.maze[self.poX - 1][self.poY] = BUMPING
                    result = 1
            # print(str(self.position.pointX)+': '+ str(self.position.pointY)+ ' ' +self.direction, action)
        elif action == TURN_RIGHT:
            if self.direction == LEFT:
                self.direction = UP
            elif self.direction == DOWN:
                self.direction = LEFT
            elif self.direction == RIGHT:
                self.direction = DOWN
            elif self.direction == UP:
                self.direction = RIGHT
        elif action == TURN_LEFT:
            if self.direction == LEFT:
                self.direction = DOWN
            elif self.direction == DOWN:
                self.direction = RIGHT
            elif self.direction == RIGHT:
                self.direction = UP
            elif self.direction == UP:
                self.direction = LEFT
        elif action == FEEL_FRONT:
            if self.direction == LEFT:
                if self.maze[self.poX][self.poY - 1] != 0:
                    result = 1
                self.maze[self.poX][self.poY - 1] = FEELING
            elif self.direction == DOWN:
                if self.maze[self.poX + 1][self.poY] != 0:
                    result = 1
                self.maze[self.poX + 1][self.poY] = FEELING
            elif self.direction == RIGHT:
                if self.maze[self.poX][self.poY + 1] != 0:
                    result = 1
                self.maze[self.poX][self.poY + 1] = FEELING
            elif self.direction == UP:
                if self.maze[self.poX - 1][self.poY] != 0:
                    result = 1
                self.maze[self.poX - 1][self.poY] = FEELING
        elif action == FEEL_LEFT:
            if self.direction == LEFT:
                if self.maze[self.poX + 1][self.poY] != 0:
                    result = 1
                self.maze[self.poX + 1][self.poY] = FEELING
            elif self.direction == DOWN:
                if self.maze[self.poX][self.poY + 1] != 0:
                    result = 1
                self.maze[self.poX][self.poY + 1] = FEELING
            elif self.direction == RIGHT:
                if self.maze[self.poX - 1][self.poY] != 0:
                    result = 1
                self.maze[self.poX - 1][self.poY] = FEELING
            elif self.direction == UP:
                if self.maze[self.poX][self.poY - 1] != 0:
                    result = 1
                self.maze[self.poX][self.poY - 1] = FEELING
        elif action == FEEL_RIGHT:
            if self.direction == LEFT:
                if self.maze[self.poX - 1][self.poY] != 0:
                    result = 1
                self.maze[self.poX - 1][self.poY] = FEELING
            elif self.direction == DOWN:
                if self.maze[self.poX][self.poY - 1] != 0:
                    result = 1
                self.maze[self.poX][self.poY - 1] = FEELING
            elif self.direction == RIGHT:
                if self.maze[self.poX + 1][self.poY] != 0:
                    result = 1
                self.maze[self.poX + 1][self.poY] = FEELING
            elif self.direction == UP:
                if self.maze[self.poX][self.poY + 1] != 0:
                    result = 1
                self.maze[self.poX][self.poY + 1] = FEELING
        # print(f"Line: {self.poX}, Column: {self.poY}, direction: {self.direction}")
        return result  
    
    def display(self):
        out.clear_output(wait=True)
        with out:
            fig, ax = plt.subplots()
            # ax.set_xticks([])
            # ax.set_yticks([])
            # ax.axis('off')
            # ax.imshow(self.maze, cmap='Greens', vmin=0, vmax=2)
            ax.imshow(self.maze, cmap=self.cmap, norm=self.norm)
            if self.direction == LEFT:
                # Y is column and X is line
                plt.scatter(self.poY, self.poX, s=200, marker='<')
            elif self.direction == DOWN:
                plt.scatter(self.poY, self.poX, s=200, marker='v')
            elif self.direction == RIGHT:
                plt.scatter(self.poY, self.poX, s=200, marker='>')
            else: # UP
                plt.scatter(self.poY, self.poX, s=200, marker='^')
            plt.show()


In [4]:
small_loop = SmallLoop(1, 1, 0)

# Le learning model

In [5]:
torch.manual_seed(0)

In [6]:
import torch
import torch.nn as nn

class FeedbackPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.input_size = input_size
        self.rnn = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, h, _):
        if h is None:
            h = torch.zeros(1, x.size(0), 16)
        out, h = self.rnn(x, h)
        return self.fc(out[:, -1, :]), h, None

# Inspired by https://github.com/LukeDitria/pytorch_tutorials.git
    
class LSTM(nn.Module):
    def __init__(self, num_emb, output_size, num_layers=1, hidden_size=128):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.num_emb = num_emb
        self.output_size = output_size

        # Create an embedding layer to convert token indices to dense vectors
        self.embedding = nn.Embedding(num_emb, hidden_size)
        
        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size=hidden_size, hidden_size=hidden_size, 
                            num_layers=num_layers, batch_first=True, dropout=0.5)
        
        # Define the output fully connected layer
        self.fc_out = nn.Linear(hidden_size, output_size)

    def forward(self, input_seq, hidden_in, mem_in):
        # Convert token indices to dense vectors
        input_embs = self.embedding(input_seq)

        # Pass the embeddings through the LSTM layer
        output, (hidden_out, mem_out) = self.lstm(input_embs, (hidden_in, mem_in))
                
        # Pass the LSTM output through the fully connected layer to get the final output
        return self.fc_out(output), hidden_out, mem_out


# L'agent

La classe Interaction

In [7]:
class Interaction:
    """An interaction is a tuple (action, outcome) with a valence"""
    def __init__(self, _action, _outcome, _valence):
        self._action = _action
        self._outcome = _outcome
        self._valence = _valence

    def get_action(self):
        """Return the action"""
        return self._action

    def get_decision(self):
        """Return the decision key"""
        return f"a{self._action}"

    def get_primitive_action(self):
        """Return the action for compatibility with CompositeInteraction"""
        return self._action

    def get_outcome(self):
        """Return the action"""
        return self._outcome

    def get_valence(self):
        """Return the action"""
        return self._valence

    def key(self):
        """ The key to find this interaction in the dictinary is the string '<action><outcome>'. """
        return f"{self._action}{self._outcome}"

    def pre_key(self):
        """Return the key. Used for compatibility with CompositeInteraction"""
        return self.key()

    def __str__(self):
        """ Print interaction in the form '<action><outcome:<valence>' for debug."""
        return f"{self._action}{self._outcome}:{self._valence}"

    def __eq__(self, other):
        """ Interactions are equal if they have the same key """
        if isinstance(other, self.__class__):
            return self.key() == other.key()
        else:
            return False

    def get_length(self):
        """The length of the sequence of this interaction"""
        return 1

In [8]:
class AgentLSTM:
    def __init__(self, interactions, model, optimizer, gap_train=11, gap_test=11, loss_func=None):
        """ 
        Création de l'agent.
        
        - self._action : action précédente
        - self._predicted_outcome : prédiction de l'outcome précédent
        """
        self._action = None
        self._predicted_outcome = None
        self._model = model
        self.optimizer = optimizer
        self._loss_func = loss_func
        self._all_outcomes = [i.get_outcome() for i in interactions]
        self._all_actions = [i.get_action() for i in interactions]
        self._history_act = []
        self._history_fb = []
        # self._valence = valence
        self._interactions = interactions
        self._gap_train = gap_train
        self._gap_test = gap_test
        self._boredom = 0

    def fit(self, actions:list, outcomes:list, nb_epoch:int=25, context_lenght=5, validate_loader=None):
        """
        Fonction d'entrainement de l'agent 
        Avec data set custom, le model prends en inputs plusieurs données
        """
        
        if len(actions) + len(outcomes) < context_lenght:
            raise Exception("Not enough data to train model")
        
        if isinstance(self._model, torch.nn.Module):
            data_loarder = CustomDataSetRNN(actions=actions, outcomes=outcomes,
                    context_lenght=context_lenght, dim_out=2, tokenizer=None)

            data_loader = torch.utils.data.DataLoader(
                data_loarder, batch_size=32, shuffle=True)
            for e in range(nb_epoch):
                for x,t in data_loader:
                    bs = t.shape[0]
                    h = torch.zeros(self._model.num_layers, bs, self._model.hidden_size, device=device)
                    cell = torch.zeros(self._model.num_layers, bs, self._model.hidden_size, device=device)

                    pred, h, cell = self._model(x, h, cell)

                    loss = self._loss_func(pred[:, -1, :], t)
                    self.optimizer.zero_grad()
                    loss.backward()
                    self.optimizer.step()
        else: # Si le model n'est pas un model pytorch
            raise Exception('Not implemented')
            self._model.fit(action, outcome)
            pass

    def predict(self, action, gap=5):
        if len(self._history_act) + len(self._history_fb) < gap:
            raise Exception("Not enough data to train model")
        x = []
        for i in range(len(self._history_act) - gap, len(self._history_act)):
            x.append(self._history_act[i])
            x.append(self._history_fb[i])
        x.append(action)
        # action = self._tokenizer.encode(x)

        action = torch.tensor([x], dtype=torch.int).to(device)

        h = torch.zeros(self._model.num_layers, 1, self._model.hidden_size, device=device)
        cell = torch.zeros(self._model.num_layers, 1, self._model.hidden_size, device=device)
        
        with torch.no_grad():  # Pas de calcul de gradients en mode prédiction
            pred, _, _ = self._model(action, h, cell)

        pred_feedback = torch.argmax(pred[:, -1, :]).item()
        # pred_feedback = self._tokenizer.decode(pred_feedback)
        
        return pred_feedback
    
    def recursif_expective_valance(self, seq:list, max_depth:int, seuil:float=0.2, proba:float = 1,
                                    seq_predi:list = []):
        """
        """
        if max_depth == 0:
            return {}
        max_depth -= 1

        if proba < seuil:
            return {}
        
        self._model.eval()
        exceptive_valance = {}
        for act in self._all_actions:
            new_seq = seq_predi + [act]
            # seq_to_predict = seq + [self._tokenizer.encode(act)]
            seq_to_predict = seq + [act]
            seq_to_predict = torch.tensor([seq_to_predict], dtype=torch.int).to(device)

            hidden = torch.zeros(self._model.num_layers, 1, self._model.hidden_size, device=device)
            memory = torch.zeros(self._model.num_layers, 1, self._model.hidden_size, device=device)

            x, _, _ = self._model(seq_to_predict, hidden, memory)
            x = x[0, -1, :]
            # Transforme x into list proba
            probs = torch.nn.functional.softmax(x, dim=0).tolist()
            # for each outcomes we want proba with act
            for i, out in enumerate(self._all_outcomes):
                tmp_new_seq = new_seq + [out]
                tmp_proba = probs[i] * proba
                if tmp_proba < seuil:
                    continue
                tempo = float(np.round(self._valence[inter(act, out)] * tmp_proba, decimals=4))
                # input(f'seq {seq_predi} act {act} out {out} proba {tmp_proba} valance {valance[(act, out)]} tempo {tempo}')
                exceptive_valance.update(
                    self.recursif_expective_valance(seq=seq[2:] + [act, out],
                                                max_depth=max_depth, seuil=seuil, 
                                                proba=tmp_proba, seq_predi=tmp_new_seq.copy())
                )
                exceptive_valance[str(tmp_new_seq)] = tempo
        return exceptive_valance
        
    def decide(self):
        x = []
        for i in range(-self._gap_test//2, 0, 1):
            x.append(self._history_act[i])
            x.append(self._history_fb[i])
        # seq = self._tokenizer.encode(x) MODIFIE
        res = self.recursif_expective_valance(seq=x, max_depth=3, seuil=0.3)
        top_5 = sorted(res.items(), key=lambda x: x[1], reverse=True)[:5]
        # print(f"Top 5 of sequences with the best expected valance for {x}")
        # for top in top_5:
        #     print(f"Sequence: {top[0]} Expected valance: {top[1]}")
        
        # print(f"Action choisie : {eval(top_5[0][0])[0]}")
        return eval(top_5[0][0])[0]

    # Modifier
    def action(self, outcome, fit=True, decide=True, validate_loader=None, force_fit=False):
        """ 
        Fonction qui choisit l'action a faire en fonction de la dernière \
        intéraction avec l'environnement. \n
        C'est ici que nous allons implémenter un mécanisme de ML \
        pour choisir la prochaine action.

        :param: **outcome** feedback de la dernière intéraction avec l'environnement

        :return: **action** action à effectuer
        """
        self._boredom += 1
        description = None
        if self._action is not None:
            self._history_fb.append(outcome)
            description = f"Action: {self._action}, Prediction: {self._predicted_outcome}, Outcome: {outcome}, \033[0;31m Satisfaction: {self._predicted_outcome == outcome} \033[0m"
            if len(self._history_act) + len(self._history_fb) > self._gap_train:
                # and self._predicted_outcome != outcome
                if fit and self._predicted_outcome != outcome:
                    self.fit(self._history_act, self._history_fb, validate_loader=validate_loader,
                            nb_epoch=50, context_lenght=self._gap_train)
                    self._boredom = 0
                
            if decide and len(self._history_act) + len(self._history_fb) > self._gap_test:
                self._action = self.decide()
            else :
                self._action = np.random.choice(self._all_actions) #  MODIFIE
                
            if len(self._history_act) + len(self._history_fb) > self._gap_test:
                self._predicted_outcome = self.predict(self._action, gap=self._gap_test)
            
            self._history_act.append(self._action)
        else:
            self._action = self._all_actions[0]
            self._history_act.append(self._action)            
            description = f"Action de base : {self._action} Prediction: {self._predicted_outcome}"
        
        return self._action, self._predicted_outcome, description

# On exécute l'agent dans le small loop

In [9]:
model_ML = LSTM(hidden_size=128, num_emb=6, num_layers=2, output_size=2)
optimizer = torch.optim.Adam(model_ML.parameters(), lr=0.001, weight_decay=0.0001)
loss_func = nn.CrossEntropyLoss()

# Instanciate the agent 
interactions = [
    Interaction(FORWARD,0,5),
    Interaction(FORWARD,1,-10),
    Interaction(TURN_LEFT,0,-3),
    Interaction(TURN_LEFT,1,-3),
    Interaction(TURN_RIGHT,0,-6),
    Interaction(TURN_RIGHT,1,-6),
    Interaction(FEEL_FRONT,0,-1),
    Interaction(FEEL_FRONT,1,-1)
]

agent_test2 = AgentLSTM(
    interactions=interactions,
    model=model_ML,
    optimizer=optimizer,
    loss_func=loss_func,
    gap_train=15,
    gap_test=15)

history_good = []
pourcent_by_10 = []
outcome = None

out = Output()
display(out)

for i in tqdm(range(50)):
    # start_time = time.time()
    action, predi, description = agent_test2.action(outcome, fit=False, decide=False)
    print(description)
    outcome = small_loop.outcome(action)
    history_good.append(outcome == predi)
    pourcent_by_10.append(sum(history_good[-10:]) * 10 if len(history_good) >= 10 else 0)
    small_loop.display()
    

Output()

  0%|          | 0/50 [00:00<?, ?it/s]

Action de base : 2 Prediction: None
Action: 2, Prediction: None, Outcome: 1,  Satisfaction: False 
Action: 3, Prediction: None, Outcome: 0,  Satisfaction: False 
Action: 5, Prediction: None, Outcome: 0,  Satisfaction: False 
Action: 5, Prediction: None, Outcome: 0,  Satisfaction: False 
Action: 2, Prediction: None, Outcome: 0,  Satisfaction: False 
Action: 2, Prediction: None, Outcome: 0,  Satisfaction: False 
Action: 3, Prediction: None, Outcome: 0,  Satisfaction: False 
Action: 2, Prediction: None, Outcome: 0,  Satisfaction: False 
Action: 5, Prediction: 1, Outcome: 0,  Satisfaction: False 
Action: 2, Prediction: 0, Outcome: 0,  Satisfaction: True 
Action: 5, Prediction: 1, Outcome: 1,  Satisfaction: True 
Action: 5, Prediction: 0, Outcome: 1,  Satisfaction: False 
Action: 2, Prediction: 0, Outcome: 1,  Satisfaction: False 
Action: 3, Prediction: 1, Outcome: 0,  Satisfaction: False 
Action: 2, Prediction: 0, Outcome: 0,  Satisfaction: True 
Action: 5, Prediction: 0, Outcome: 0,  Sati